In [2]:
import pandas as pd
import requests
from pytz import timezone
from datetime import datetime
import sqlalchemy

In [10]:
schema = "sql_workshop"
host = "34.77.20.25"
user = "root"
password = "sushma25121998"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## Reading cities table from sql

In [12]:
cities_df = pd.read_sql("cities", con=connection_string)
cities_df

,city_id,city,country,latitude,longitude


In [21]:
berlin_timezone = timezone('Europe/Berlin')
API_key = 'ec3f32171441d6fdef6cab30f523db6b'
weather_items = []

In [31]:
# For loop over the "City_name" column in the "cities_df"

for i in cities_df["city"]:
    print(i)

Berlin
Hamburg
Munich
Cologne
Amsterdam
Paris


In [35]:
# Extracting the "City_id" from the cities table. This will be implemented in the for loop.
cities_df.loc[cities_df["city"] == "Hamburg", "city_id"].values[0]

2

In [37]:
for city in cities_df["city"]:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()
    # As we are now using the data from our relational database
    # the city should reflect the city_id and not the city name
    city_id = cities_df.loc[cities_df["city"] == city, "city_id"].values[0]

    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    for item in json["list"]:
        weather_item = {
            "city_id": city_id,
            "forecast_time": item.get("dt_txt", None),
            "outlook": item["weather"][0].get("description", None),
            "temperature": item["main"].get("temp", None),
            "feels_like": item["main"].get("feels_like", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "wind_speed": item["wind"].get("speed", None),
            "rain_prob": item.get("pop", None),
            "data_retrieved_at": retrieval_time
        }

        weather_items.append(weather_item)

In [39]:
weather_df = pd.DataFrame(weather_items)

In [41]:
weather_df

,city_id,forecast_time,outlook,temperature,feels_like,rain_in_last_3h,wind_speed,rain_prob,data_retrieved_at
0,1,2024-09-10 15:00:00,broken clouds,19.54,18.86,0.0,5.66,0.0,2024-09-10 14:19:50
1,1,2024-09-10 18:00:00,broken clouds,18.31,17.68,0.0,3.80,0.0,2024-09-10 14:19:50
2,1,2024-09-10 21:00:00,broken clouds,17.03,16.43,0.0,4.25,0.0,2024-09-10 14:19:50
3,1,2024-09-11 00:00:00,overcast clouds,17.06,16.39,0.0,4.67,0.0,2024-09-10 14:19:50
4,1,2024-09-11 03:00:00,overcast clouds,15.71,15.14,0.0,5.66,0.0,2024-09-10 14:19:50
...,...,...,...,...,...,...,...,...,...
235,6,2024-09-15 00:00:00,broken clouds,13.22,12.06,0.0,1.94,0.0,2024-09-10 14:19:51
236,6,2024-09-15 03:00:00,overcast clouds,12.16,11.08,0.0,1.38,0.0,2024-09-10 14:19:51
237,6,2024-09-15 06:00:00,overcast clouds,11.54,10.37,0.0,1.30,0.0,2024-09-10 14:19:51
238,6,2024-09-15 09:00:00,clear sky,15.35,14.17,0.0,2.41,0.0,2024-09-10 14:19:51


In [43]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   city_id            240 non-null    int64  
 1   forecast_time      240 non-null    object 
 2   outlook            240 non-null    object 
 3   temperature        240 non-null    float64
 4   feels_like         240 non-null    float64
 5   rain_in_last_3h    240 non-null    float64
 6   wind_speed         240 non-null    float64
 7   rain_prob          240 non-null    float64
 8   data_retrieved_at  240 non-null    object 
dtypes: float64(5), int64(1), object(3)
memory usage: 17.0+ KB


In [45]:
weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

In [47]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   city_id            240 non-null    int64         
 1   forecast_time      240 non-null    datetime64[ns]
 2   outlook            240 non-null    object        
 3   temperature        240 non-null    float64       
 4   feels_like         240 non-null    float64       
 5   rain_in_last_3h    240 non-null    float64       
 6   wind_speed         240 non-null    float64       
 7   rain_prob          240 non-null    float64       
 8   data_retrieved_at  240 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(5), int64(1), object(1)
memory usage: 17.0+ KB


In [60]:
weather_df.to_sql('weather',
                  if_exists='append',
                  con=connection_string,
                  index=False)

240